In [1]:
%load_ext google.cloud.bigquery

# "Will it snow tomorrow?" - The time traveler asked
The following dataset contains climate information from over 9000 stations accross the world. The overall goal of these subtasks will be to predict whether it will snow tomorrow 14 years ago. So if today is 2023.10.27 then the weather we want to forecast is for the date 2009.10.28. You are supposed to solve the tasks using Big Query, which can be used in the Jupyter Notebook like it is shown in the following cell. For further information and how to use BigQuery in Jupyter Notebook refer to the Google Docs. 

The goal of this test is to test your coding knowledge in Python, BigQuery and Pandas as well as your understanding of Data Science. If you get stuck in the first part, you can use the replacement data provided in the second part

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="../sevenlearnings-datascience-f06eb42a7306.json"

## Part 1

### 1. Task
Change the date format to 'YYYY-MM-DD' and select the data from 2005 till 2009 for station numbers including and between 725300 and 726300 , and save it as a pandas dataframe. Note the maximum year available is 2010. 

In [3]:
%%bigquery df
SELECT 
    *, 
    PARSE_DATE('%Y-%m-%d', CONCAT(CAST(year AS STRING), '-', CAST(month AS STRING), '-', CAST(day AS STRING))) AS date
FROM 
    bigquery-public-data.samples.gsod
WHERE
    year BETWEEN 2005 AND 2009 
    AND station_number BETWEEN 725300 AND 726300

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 377784/377784 [00:12<00:00, 29764.64rows/s]


### 2. Task 
From here you want to work with the data from all stations 725300 to 725330 that have information from 2005 till 2009. 

Do a first analysis of the remaining dataset, clean or drop data depending on how you see appropriate. 

**Nacho -** for a quick exploration and decide on what to drop/transform I:
 - Check if the schema brings some docs/descriptions of the columns (it does not)
 - Use `pd.describe()` to have a feeling of the values of both numerical and categorical/bool cols
 - Check the ratio of uniques and nulls

In [4]:
%%bigquery
-- looking for column descriptions

SELECT
    *
FROM
    bigquery-public-data.samples.INFORMATION_SCHEMA.COLUMNS
WHERE
    table_name="gsod"
LIMIT 1

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/rows]


,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_generated,generation_expression,is_stored,is_hidden,is_updatable,is_system_defined,is_partitioning_column,clustering_ordinal_position,collation_name,column_default,rounding_mode
0,bigquery-public-data,samples,gsod,station_number,1,NO,INT64,NEVER,None,None,NO,None,NO,NO,NaN,NULL,NULL,None


In [5]:
import pandas as pd
import numpy as np

# overviewing features (I don't print it for space)
df.describe(include=np.number).T
df.describe(exclude=np.number).T


# checking uniques and nulls
unique_and_nulls = {}

for col in df:
    uniques = len(df[col].unique())
    nulls = df[col].isnull().sum()
    null_percentage = round((nulls / len(df)) * 100)

    unique_and_nulls[col] = {
        "uniques": uniques,
        "nulls": nulls,
        "null_percentage": null_percentage
    }

pd.DataFrame(unique_and_nulls).T

,uniques,nulls,null_percentage
station_number,217,0,0
wban_number,204,0,0
year,5,0,0
month,12,0,0
day,31,0,0
mean_temp,1139,0,0
num_mean_temp_samples,21,0,0
mean_dew_point,1050,158,0
num_mean_dew_point_samples,22,158,0
mean_sealevel_pressure,648,145847,39


**Nacho -** at first glance:
 - We don't need year, month and date because that info is passed in the new date
 - There are several columns filled >95% with nulls we can drop
 - Presumably the number of samples used for the means are not necessary either
 - Looks like wban_number is not usable. Arguably station_number could hold some geographical info. However, it looks almost as a categorical value (although encoded as a number) and I will drop it too. Theres another public table in BQ with station info, so I could eventually take latitude, longitude and elevation with something like the query below (I won't do it):

```SQL
with nb_gsod as(
  SELECT 
    *,
    CAST(station_number as string) as usaf,
    PARSE_DATE('%Y-%m-%d', CONCAT(CAST(year AS STRING), '-', CAST(month AS STRING), '-', CAST(day AS STRING))) AS date
  FROM 
    bigquery-public-data.samples.gsod
  WHERE
    year BETWEEN 2005 AND 2009 
    AND station_number BETWEEN 725300 AND 726300
),

stations as (
  select usaf, lat, lon, elev
  from `bigquery-public-data.noaa_gsod.stations`
)

select *
from
  nb_gsod
  left join stations on nb_gsod.usaf=stations.usaf
```

In [6]:
df.drop(
    columns=[
        "year",
        "month",
        "day",
        "min_temperature",
        "mean_station_pressure",
        "num_mean_station_pressure_samples",
        "num_mean_temp_samples",
        "num_mean_dew_point_samples",
        "num_mean_sealevel_pressure_samples",
        "num_mean_visibility_samples",
        "min_temperature_explicit",
        "snow_depth", # this could stay
        "num_mean_wind_speed_samples",
        "wban_number",
        "station_number",
    ],
    inplace=True,
)

**Nacho** - with the cleaner table I can start transforming:
 - Encoding bools (fog, rain, etc.) - I'd go for easy binary encoding
 - Encoding dates - I'll use values 1 to 365 and apply cosine encoding
 - Treating remaining mulls - Looks like only 2% of the rows have >2 nulls (out of 14 features). I'd assume Normal distribution and impute the mean for numerical and mode for booleans (after train/test split to avoid leakage).
 - Outlier analysis and imputation could also be done, but I don't think is necessary

In [7]:
# binary encode bools
bool_cols = [col for col in df.columns if df[col].dtype == bool]
bool_cols.append("max_temperature_explicit") # was object type but filled with bools

for col in bool_cols:
    df[col] = df[col].replace({True:1, False:0})
    

# cyclic encode date
df['date'] = pd.to_datetime(df['date'])
df['day_of_year'] = df['date'].dt.dayofyear
    
def encode_cyclical(day: int) -> float:
    return (np.cos(2 * np.pi * day / 366) + 1) / 2

df['cyclic_date'] = df.apply(lambda row: encode_cyclical(row['day_of_year']), axis=1)


# checking rows with >x nulls
null_rows = df.isnull().sum(axis=1)
for i in range(7):
    print(f"% of rows with >{i} nulls: {len(null_rows[null_rows > i])/len(df)}")
    

# split data before imputation
from sklearn.model_selection import train_test_split

test_day = "2009-11-04"
test = df[df['date'] == test_day]
df = df[df['date'] < test_day] # I should not have data after "tomorrow"

train, val = train_test_split(df, test_size=0.2, random_state=42) 


# I don't need not-encoded dates anymore
for split in [train, val, test]:
    split.drop(columns=["date", "day_of_year"], inplace=True)
    

# imput training set statistics
train_modes = train[bool_cols].mode().iloc[0]
num_cols = [col for col in df.columns if df[col].dtype == float]
train_means = train[num_cols].mean()

for split in [train, val, test]:
    for col in split.columns:
        if col in bool_cols:
            split[col].fillna(train_modes[col], inplace=True)
        elif col in num_cols:
            split[col].fillna(train_means[col], inplace=True)
            
train.head()

% of rows with >0 nulls: 0.6463561188403956
% of rows with >1 nulls: 0.2118670986595515
% of rows with >2 nulls: 0.02373843254346399
% of rows with >3 nulls: 0.0009105732376172628
% of rows with >4 nulls: 8.470448722021049e-05
% of rows with >5 nulls: 1.0588060902526311e-05
% of rows with >6 nulls: 1.0588060902526311e-05


,mean_temp,mean_dew_point,mean_sealevel_pressure,mean_visibility,mean_wind_speed,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,max_temperature_explicit,total_precipitation,fog,rain,snow,hail,thunder,tornado,cyclic_date
294254,64.000000,37.799999,1011.500000,10.0,8.4,18.1,28.900000,44.099998,0.0,0.0000,0,0,0,0,0,0,0.135986
44644,35.299999,21.700001,1016.265921,10.0,9.3,25.1,29.900000,26.600000,1.0,0.0667,0,0,0,0,0,0,0.444322
277482,56.099998,53.200001,1018.299988,8.6,3.1,8.0,23.515039,53.599998,1.0,0.0400,0,0,0,0,0,0,0.026363
46800,22.400000,18.000000,1016.265921,8.7,4.3,15.9,22.900000,19.400000,1.0,0.0000,0,0,0,0,0,0,0.983514
219082,42.000000,31.500000,1016.265921,13.4,11.9,26.0,35.000000,37.400002,1.0,0.0667,0,0,0,0,0,0,0.265012


**Nacho** - looking good now. Finally I:
 - Analyze correlation - drop three cols above 0.8
 - Scale the features - I fit a standard scaler on the training set and transform all splits with that

In [8]:
# remove correlated cols
corr_matrix = train[num_cols].corr()
threshold = 0.8

mask = np.ones(corr_matrix.shape)
mask = np.triu(mask, k=1).astype(bool)
upper = corr_matrix.where(mask)
correlated_cols = [column for column in upper.columns if any(upper[column] > threshold)]

for split in [train, val, test]:
    split.drop(columns = correlated_cols, inplace=True)
    

# grab binary target and scale the rest
from sklearn.preprocessing import StandardScaler

y_train = train["snow"]
y_test = test["snow"]
y_val = val["snow"]

x_train = train.drop('snow', axis=1) 
x_test = test.drop('snow', axis=1)
x_val = val.drop('snow', axis=1)

scaler = StandardScaler()
scaler.fit(x_train)

x_train = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns, index=x_train.index)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_train.columns, index=x_test.index)
x_val = pd.DataFrame(scaler.transform(x_val), columns=x_train.columns, index=x_val.index)

### 3. Task
Now it is time to split the data, into a training, evaluation and test set. As a reminder, the date we are trying to predict snow fall for should constitute your test set.

In [9]:
import datetime as dt

str(dt.datetime.today()- dt.timedelta(days=14*365)).split(' ')[0]

'2009-11-06'

**Nacho -** I made it above, before null imputation

## Part 2
If you made it up to here all by yourself, you can use your prepared dataset to train an algorithm of your choice to forecast whether it will snow on the following date for each station in this dataset:

In [10]:
import datetime as dt

str(dt.datetime.today()- dt.timedelta(days=14*365)).split(' ')[0]

'2009-11-06'

You are allowed to use any library you are comfortable with such as sklearn, tensorflow, keras etc. 
If you did not manage to finish part one feel free to use the data provided in 'coding_challenge.csv' Note that this data does not represent a solution to Part 1. 

**Nacho** - a simple random forest achieves perfect scores 🤷. Same for a shallow neural network on just 2 epochs. Since most rows have value `False` for the target variable, I'll go with F1score as main metric.

In [11]:
from sklearn.metrics import precision_score, recall_score, f1_score

def compute_metrics(y_true: list, y_pred: list) -> dict:
    return {
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
    }

In [12]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x_train, y_train)

test_preds = rf.predict(x_test)
compute_metrics(y_test, test_preds)

{'precision': 1.0, 'recall': 1.0, 'f1': 1.0}

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.metrics import Precision, Recall

nn = Sequential()
nn.add(Dense(32, activation='relu', input_dim=x_train.shape[1]))
nn.add(Dense(1, activation='sigmoid'))
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Precision(), Recall()])

nn.fit(x_train, y_train, epochs=2, batch_size=32, validation_data=(x_val, y_val))

test_preds = nn.predict(x_test)
test_preds = [1 if i > 0.5 else 0 for i in test_preds]
compute_metrics(y_test, test_preds)

Epoch 1/2
9145/9145 [==============================] - 27s 3ms/step - loss: 0.0055 - precision: 0.9986 - recall: 0.9959 - val_loss: 3.0659e-06 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 2/2
7/7 [==============================] - 0s 2ms/step


{'precision': 1.0, 'recall': 1.0, 'f1': 1.0}